#LOW-LIGHT + NUCLEAR NORM

Oscar Miguel Ortega Lozano - 2220528
Daniel Esteban Yaruro Contreras - 2220088
Juan David Toloza Parada - 2221136

---
## Problema de Imagen en Condiciones de Baja Iluminación (Low-Light) usando ADMM con Regularización por Norma Nuclear

Este notebook presenta una implementación del algoritmo **ADMM (Alternating Direction Method of Multipliers)** para resolver un problema de mejora de imagen en condiciones de **baja iluminación**. Se modela el ruido como una distribución de **Poisson**, y se usa la **norma nuclear** como prior para regularización.

---

### 1. Formulación del problema

Dado:
- Imagen observada $y$ con ruido tipo Poisson.
- Imagen original desconocida $x$.

Modelo probabilístico:
$y \sim \mathcal{P}(yx)$

Función objetivo (verosimilitud negativa + regularización):
$$
\min_x \quad y^T \cdot \mathbf{1} - y^T \log(yx) + \lambda \|x\|_*
$$

Donde:
- $\|x\|_*$ es la **norma nuclear** (suma de valores singulares de $x$).
- $\lambda$ es un parámetro que controla la regularización.

---

### 2. Algoritmo ADMM

Reformulamos con una variable auxiliar $z$:
$$
\min_{x, z} \quad f(x) + g(z) \quad \text{sujeto a } x = z
$$
Donde:
- $f(x) = y^T \cdot \mathbf{1} - y^T \log(yx)$
- $g(z) = \lambda \|z\|_*$

#### Pasos del algoritmo ADMM:
Inicializar $x^0, z^0, u^0$. Luego iterar:

1. **Actualización de x:**
$$
    x^{k+1} = \arg\min_x \left[ f(x) + \frac{\rho}{2} \|x - z^k + u^k\|_2^2 \right]
$$

2. **Actualización de z:**
$$
    z^{k+1} = \arg\min_z \left[ g(z) + \frac{\rho}{2} \|x^{k+1} - z + u^k\|_2^2 \right]
$$
Esta es la **proximidad de la norma nuclear**:
$$
    z = \text{prox}_{\lambda/\rho}(x + u) = U \cdot \text{soft}(\Sigma, \lambda/\rho) \cdot V^T
$$
Donde:
- $x + u = U \Sigma V^T$ es la descomposición SVD.
- $\text{soft}(\Sigma, t) = \max(\Sigma - t, 0)$

3. **Actualización del multiplicador dual:**
$$
    u^{k+1} = u^k + x^{k+1} - z^{k+1}
$$

---

In [ ]:

### 3. Código Python

```python
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# -------------------------------
# Funciones auxiliares
# -------------------------------

def add_poisson_noise(image):
    noisy = np.random.poisson(image * 255) / 255.0
    return np.clip(noisy, 0, 1)

def nuclear_prox(X, tau):
    U, S, VT = np.linalg.svd(X, full_matrices=False)
    S_soft = np.maximum(S - tau, 0)
    return U @ np.diag(S_soft) @ VT

def poisson_loss(x, y):
    x = np.clip(x, 1e-4, 1)
    return np.sum(y - y * np.log(x))

# -------------------------------
# ADMM para Low-Light + Nuclear Norm
# -------------------------------

def admm_lowlight_nuclear(y, lamb=0.1, rho=1, max_iter=50):
    x = y.copy()
    z = y.copy()
    u = np.zeros_like(y)

    for i in range(max_iter):
        # x-update (solución cerrada usando Newton o gradiente descendente)
        x = (rho * (z - u) + y) / (rho + y)

        # z-update (prox de nuclear norm)
        z = nuclear_prox(x + u, lamb / rho)

        # u-update
        u = u + x - z

    return x

# -------------------------------
# Cargar imagen y procesar
# -------------------------------

# Cargar imagen en escala de grises y normalizar
img = cv2.imread('tu_imagen.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (256, 256))
img = img / 255.0

# Agregar ruido Poisson
noisy_img = add_poisson_noise(img)

# Aplicar ADMM
restored_img = admm_lowlight_nuclear(noisy_img, lamb=0.2, rho=1, max_iter=50)

# Mostrar resultados
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title('Original')
plt.imshow(img, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('Noisy (Low-Light)')
plt.imshow(noisy_img, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Restored (ADMM)')
plt.imshow(restored_img, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()
```
